# Modello LSMT (Long Short-Term Memory) 

Primo step split del dataset

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import joblib
from collections import Counter
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, RepeatVector
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

INPUT_DIR = '../Pre-Elaborazione Dati/Dataset' 
SCALER_PATH = 'scaler.joblib' 
COLONNE_FEATURES = ['Latitude', 'Longitude', 'SOG', 'COG']

WINDOW_SIZE = 30  
BATCH_SIZE = 64   

all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))
TRAIN_FILES = all_files[0:16]
VAL_FILES = all_files[16:20]
TEST_FILES = all_files[20:24] 

print("Configurazione caricata.")
print(f"Window Size: {WINDOW_SIZE}, Batch Size: {BATCH_SIZE}")

In [ ]:
all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

if len(all_files) == 0:
    print(f"ERRORE CRITICO: Nessun file .parquet trovato in '{INPUT_DIR}'")
else:
    print(f"Trovati {len(all_files)} file .parquet in '{INPUT_DIR}'")

TRAIN_FILES = all_files[0:16]
IPERPARAMETRI_TIMESTEP = [10, 15, 20, 25, 30, 40]

print(f"\nFASE 1: Avvio scansione unificata (UN FILE ALLA VOLTA)...")

total_counts_counter = Counter()
total_ids_set = set()

for file_path in TRAIN_FILES:
    print(f"  Processando {os.path.basename(file_path)}...")
    try:
        df = pd.read_parquet(file_path, columns=['TrajectoryID'])
        
        total_ids_set.update(df['TrajectoryID'].unique())
        
        total_counts_counter.update(df['TrajectoryID'])
        
        del df
        gc.collect()
    except Exception as e:
        print(f"    ERRORE nel leggere {file_path}: {e}")
        

print("FASE 2: Verifica e Analisi.")

conteggio_set = len(total_ids_set)
conteggio_counter = len(total_counts_counter)

print(f"\VERIFICA CONTEGGI:")
print(f"ID Unici (metodo 'set'):     {conteggio_set:,}")
print(f"ID Unici (metodo 'Counter'): {conteggio_counter:,}")

if conteggio_set != conteggio_counter:
    print("ERRORE LOGICO: I conteggi non corrispondono. C'è ancora un problema.")
else:
    print("\nSUCCESSO! I conteggi corrispondono.")
    
    all_lengths = list(total_counts_counter.values())
    totale_traiettorie_analisi = conteggio_counter

    print(f"\nRisultati dell'analisi (su {totale_traiettorie_analisi:,} traiettorie totali di TRAINING):")
    print("-" * 75)
    print(f"{'Window Size':<15} | {'Traiettorie Usabili':<20} | {'Traiettorie Scartate':<20} | {'% Usabili':<10}")
    print("-" * 75)

    for timestep in IPERPARAMETRI_TIMESTEP:
        traiettorie_usabili = sum(1 for length in all_lengths if length >= timestep)
        traiettorie_scartate = totale_traiettorie_analisi - traiettorie_usabili
        percentuale_usabili = (traiettorie_usabili / totale_traiettorie_analisi) * 100
        print(f"{timestep:<15} | {traiettorie_usabili:<20} | {traiettorie_scartate:<20} | {percentuale_usabili:<10.2f}%")

    print("-" * 75)
    print("Analisi completata.")

<>:46: SyntaxWarning: invalid escape sequence '\V'
<>:46: SyntaxWarning: invalid escape sequence '\V'
/tmp/ipykernel_19344/47474260.py:46: SyntaxWarning: invalid escape sequence '\V'
  print(f"\VERIFICA CONTEGGI:")


Trovati 25 file .parquet in '../Pre-Elaborazione Dati/Dataset'

FASE 1: Avvio scansione unificata (UN FILE ALLA VOLTA)...
  Processando blocco_000-segmentato.parquet...
  Processando blocco_001-segmentato.parquet...
  Processando blocco_002-segmentato.parquet...
  Processando blocco_003-segmentato.parquet...
  Processando blocco_004-segmentato.parquet...
  Processando blocco_005-segmentato.parquet...
  Processando blocco_006-segmentato.parquet...
  Processando blocco_007-segmentato.parquet...
  Processando blocco_008-segmentato.parquet...
  Processando blocco_009-segmentato.parquet...
  Processando blocco_010-segmentato.parquet...
  Processando blocco_011-segmentato.parquet...
  Processando blocco_012-segmentato.parquet...
  Processando blocco_013-segmentato.parquet...
  Processando blocco_014-segmentato.parquet...
  Processando blocco_015-segmentato.parquet...

------------------------------------------------------------
FASE 1: Scansione completata.
FASE 2: Verifica e Analisi.
\VERIF

### Normalizzazione

Passaggio fondamentale perchè le tue reti neurali (LSTM/LNN) faticano a imparare quando i dati di input hanno scale completamente diverse. Ad esempio, la colonna COG (rotta) va da 0 a 360, mentre la colonna SOG (velocità) potrebbe andare da 0 a 50. Questi intervalli così diversi "confondono" la rete durante l'addestramento.
Dobbiamo quindi standardizzarli, portando tutte le colonne a una scala simile.  
Il modo corretto per standardizzare è calcolare la media e la deviazione standard dell'intero set di addestramento (tutti i 4.3 milioni di traiettorie nei tuoi 16 file). Ovviamente, questi dati sono troppo grandi per essere caricati tutti insieme in memoria.

Quello che facciamo è:  
- Inizializziamo uno scaler vuoto e iteriamo sui 16 file di Train.
- `partial_fit` è la parte fondamentale perchè per ogni file che carica. Questa funzione dice allo scaler: "Prendi la media e la deviazione che hai calcolato finora e aggiornale includendo questo nuovo blocco di dati".
- `joblib.dump(scaler, SCALER_PATH)` salva questi parametri calcolati in un file esterno.


In [5]:
COLONNE_DA_NORMALIZZARE = ['Latitude', 'Longitude', 'SOG', 'COG']
SCALER_PATH = 'scaler.joblib'

scaler = StandardScaler()

try:
    
    for i, file_path in enumerate(TRAIN_FILES):
        
        print(f"Processando file {i+1}/{len(TRAIN_FILES)}: {os.path.basename(file_path)}")
        
        df_chunk = pd.read_parquet(file_path, columns=COLONNE_DA_NORMALIZZARE)
        
        scaler.partial_fit(df_chunk)
        
        del df_chunk
        gc.collect()

    print("\nAdattamento completato su tutti i 16 file di training.")
    
    joblib.dump(scaler, SCALER_PATH) #Salva file

    print(f"\nScaler salvato come '{SCALER_PATH}'.")

except Exception as e:
    print(f"\n Errore durante la preparazione dello scaler: {e}")

Processando file 1/16: blocco_000-segmentato.parquet
Processando file 2/16: blocco_001-segmentato.parquet
Processando file 3/16: blocco_002-segmentato.parquet
Processando file 4/16: blocco_003-segmentato.parquet
Processando file 5/16: blocco_004-segmentato.parquet
Processando file 6/16: blocco_005-segmentato.parquet
Processando file 7/16: blocco_006-segmentato.parquet
Processando file 8/16: blocco_007-segmentato.parquet
Processando file 9/16: blocco_008-segmentato.parquet
Processando file 10/16: blocco_009-segmentato.parquet
Processando file 11/16: blocco_010-segmentato.parquet
Processando file 12/16: blocco_011-segmentato.parquet
Processando file 13/16: blocco_012-segmentato.parquet
Processando file 14/16: blocco_013-segmentato.parquet
Processando file 15/16: blocco_014-segmentato.parquet
Processando file 16/16: blocco_015-segmentato.parquet

Adattamento completato su tutti i 16 file di training.

Scaler salvato come 'scaler.joblib'.


#### Funzioni del Data Generator

In [ ]:
def calculate_steps_per_epoch(file_paths, window_size, batch_size):
    """
    Scansiona tutti i file in modo efficiente (RAM-safe) per calcolare
    il numero totale di finestre (campioni) che verranno generate,
    e da lì calcola il numero di "steps" (batch) per epoca.
    Usa il metodo Counter.update(Series) che sappiamo funzionare.
    """
    print(f"--- Calcolo Steps per {len(file_paths)} file ---")
    
    total_lengths = Counter()
    
    # FASE 1: Trova le lunghezze di tutte le 4.3M traiettorie
    for i, file_path in enumerate(file_paths):
        print(f"  Scansione lunghezze file {i+1}/{len(file_paths)}...", end='\r')
        try:
            df = pd.read_parquet(file_path, columns=['TrajectoryID'])
            # Metodo efficiente (provato) per aggiornare le lunghezze
            total_lengths.update(df['TrajectoryID'])
            del df
            gc.collect()
        except Exception as e:
            print(f"Errore nel leggere {file_path}: {e}")

    print("\nScansione lunghezze completata.")

    # FASE 2: Calcola il numero totale di finestre
    total_windows = 0
    for length in total_lengths.values():
        if length >= window_size:
            # Una traiettoria di N righe produce (N - window_size + 1) finestre
            total_windows += (length - window_size + 1)
            
    # FASE 3: Calcola gli steps
    steps = int(np.ceil(total_windows / batch_size))
    
    print(f"Trovate {total_windows:,} finestre totali.")
    print(f"Steps per Epoca (Batch Size {batch_size}): {steps}")
    print("------------------------------------------")
    return steps






def create_windows(data_np, window_size):
    """
    Crea finestre mobili da un array numpy 2D.
    """
    windows = []
    # Itera da 0 fino all'ultimo punto di inizio possibile
    for i in range(len(data_np) - window_size + 1):
        windows.append(data_np[i : i + window_size])
    return windows





def data_generator(file_paths, scaler, features, window_size, batch_size, shuffle_files=True):
    """
    Generatore (yield) che carica i file uno per uno, gestisce la 
    RAM e "cuce" le traiettorie che si trovano a cavallo dei file.
    """
    
    # Questo buffer tiene in memoria le "code" delle traiettorie
    # alla fine di un file, per cucirle con l'inizio del file successivo.
    # Formato: { 'TrajectoryID': DataFrame_coda }
    trajectory_buffer = {}
    
    # Questo buffer tiene le finestre pronte per essere messe in un batch
    window_buffer = []
    
    # Loop infinito: Keras chiamerà questo generatore più volte
    while True:
        
        # Opzionale: mescola l'ordine dei file ad ogni epoca
        if shuffle_files:
            np.random.shuffle(file_paths)
            
        # Itera sui file di training (o validazione)
        for file_path in file_paths:
            try:
                # Carica UN file in memoria
                df = pd.read_parquet(file_path)
                
                # Normalizza i dati USANDO LO SCALER CARICATO
                df[features] = scaler.transform(df[features])
                
                # Questo buffer terrà le code del file *corrente*
                # da passare al file successivo
                next_file_buffer = {}

                # Itera sui gruppi (questo è RAM-efficiente)
                # Grazie all'ordinamento, i gruppi sono contigui
                for tid, group in df.groupby('TrajectoryID'):
                    
                    # --- 1. CUCITURA ---
                    # Controlla se questo ID era già nel buffer (dal file PRECEDENTE)
                    if tid in trajectory_buffer:
                        # Cucitura!
                        trajectory_data = pd.concat([trajectory_buffer.pop(tid), group])
                    else:
                        # Nessuna cucitura, è una nuova traiettoria
                        trajectory_data = group
                        
                    # --- 2. FILTRO LUNGHEZZA ---
                    if len(trajectory_data) < window_size:
                        # Troppo corta PER ORA. Mettila da parte
                        next_file_buffer[tid] = trajectory_data
                        continue # Passa al prossimo TrajectoryID
                        
                    # --- 3. CREAZIONE FINESTRE ---
                    # Converti in NumPy per velocità
                    trajectory_np = trajectory_data[features].to_numpy()
                    
                    # Crea tutte le finestre possibili da questa traiettoria
                    new_windows = create_windows(trajectory_np, window_size)
                    
                    # Aggiungi le finestre al buffer dei batch
                    window_buffer.extend(new_windows)
                    
                    # --- 4. SALVATAGGIO CODA PER PROSSIMO FILE ---
                    # Salva le ultime (N-1) righe. Saranno la base
                    # per la cucitura nel *prossimo* file.
                    next_file_buffer[tid] = trajectory_data.iloc[-(window_size - 1):]

                    # --- 5. PRODUZIONE (YIELD) BATCH ---
                    # Abbiamo abbastanza finestre per un batch?
                    while len(window_buffer) >= batch_size:
                        
                        # Estrai il batch
                        batch_to_yield = window_buffer[:batch_size]
                        # Rimuovilo dal buffer
                        window_buffer = window_buffer[batch_size:]
                        
                        # Converti in array numpy finale
                        batch_np = np.array(batch_to_yield)
                        
                        # L'output (y) è uguale all'input (X)
                        yield (batch_np, batch_np)
                
                # Finito il file, aggiorna il buffer principale
                # per il *prossimo* file
                trajectory_buffer = next_file_buffer

            except Exception as e:
                print(f"\nERRORE nel generatore durante la lettura di {file_path}: {e}")
                # Continua con il file successivo
                continue

print("Funzioni definite: calculate_steps_per_epoch, create_windows, data_generator")

#### Calcolo degli steps

In [ ]:
print("Avvio calcolo steps per il Training Set...")
train_steps = calculate_steps_per_epoch(TRAIN_FILES, WINDOW_SIZE, BATCH_SIZE)

print("\nAvvio calcolo steps per il Validation Set...")
val_steps = calculate_steps_per_epoch(VAL_FILES, WINDOW_SIZE, BATCH_SIZE)

print("\n--- Calcolo Steps Completato ---")
print(f"Training Steps per Epoca: {train_steps}")
print(f"Validation Steps per Epoca: {val_steps}")

#### Caricamento Scaler e creazione dei Generatori

In [ ]:
# --- B. Carica lo Scaler ---
print(f"Caricamento scaler da {SCALER_PATH}...")
scaler = joblib.load(SCALER_PATH)

# --- C. Inizializza i Generatori ---
print("Inizializzazione dei generatori (oggetti pronti)...")

train_gen = data_generator(
    file_paths=TRAIN_FILES,
    scaler=scaler,
    features=COLONNE_FEATURES,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    shuffle_files=True
)

val_gen = data_generator(
    file_paths=VAL_FILES,
    scaler=scaler,
    features=COLONNE_FEATURES,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    shuffle_files=False
)

print("Generatori pronti.")

#### Definizione Modello LSMT Autoencoder

In [ ]:
# --- D. Definisci il Modello (LSTM Autoencoder) ---

n_features = len(COLONNE_FEATURES)
latent_dim = 32 # Prova a cambiare questo iperparametro (es. 64, 128)

# Encoder
inputs = Input(shape=(WINDOW_SIZE, n_features))
# Il primo LSTM comprime l'input
lstm_encoder = LSTM(latent_dim, return_sequences=False)(inputs)

# Decoder
# Ripete il vettore compresso per ogni timestep
repeat_vector = RepeatVector(WINDOW_SIZE)(lstm_encoder)
# Il secondo LSTM "legge" il vettore compresso e ricostruisce la sequenza
lstm_decoder = LSTM(latent_dim, return_sequences=True)(repeat_vector)
# Un layer finale per rimappare all'output (n_features)
output = TimeDistributed(Dense(n_features))(lstm_decoder)

model_lstm = Model(inputs, output)
model_lstm.compile(optimizer='adam', loss='mae') # MAE è ottimo per questo

print("Modello LSTM-Autoencoder creato e compilato.")
model_lstm.summary()

#### Addestramento LSMT

In [ ]:
# --- E. Avvia l'Addestramento ---

# Definisci i Callback
checkpoint = ModelCheckpoint(
    'lstm_autoencoder_best.h5', # Salva il modello migliore
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5, # Ferma se non migliora per 5 epoche
    mode='min',
    verbose=1,
    restore_best_weights=True
)


print(f"\nAvvio addestramento LSTM-Autoencoder...")

history_lstm = model_lstm.fit(
    train_gen,
    steps_per_epoch=train_steps,
    epochs=50, # Metti un numero alto, EarlyStopping lo fermerà
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=[checkpoint, early_stopping]
)

print("\nAddestramento LSTM completato.")